<a href="https://colab.research.google.com/github/taylan-sen/CIS320_SP24_pub/blob/main/Networking_TCP_%26_HTTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###In this notebook, we will go over:
  1. How to use python jupyter notebooks as a python coding and sharing environment.
  1. Show how to make a TCP connection.
  1. Use TCP to interact directly with a webserver.

Please complete the following activities. It will be the first steps to project 2.

### 1. jupyter notebooks
**Google colab** is a google's free virtual machine service implementating a **jupyter** notebook environment. *jupyter notebooks* use a web browser to run run python code, save the results, as well as provide the ability to document code and results nicely.

Jupyter notebooks have the .ipynb extension. A notebook is made up of vertically stacked **cells**. Cells can be **code** cells, which contain python code, or **text** cells, which contain text written in **markdown**. **Markdown** is like a *light* version of html, it lets you insert some text formatting and links without getting bogged down with too much html tag-like gobbledygook.

A colab jupyter notebook is running in its own google-managed VM (you can see the RAM and disk allocated in the top left of a connected notebook). In order to run a code cell, click on the *play* button in the top left of the cell.

In order to launch your own google colab jupyter notebook, log into your google account in a browser and go to:

http://colab.research.google.com

The first time you use colab, you may have to *enable it*. After opening your own notebook, try making a code cell to print hello world as shown below:

In [ ]:
print("Hello world!")

Hello world!


Colab jupyter notebooks can also be used to send out and receive transport layer packets using the python socket module. In the code below, we send a message to our infamous mothership. Consider the following socket code we learned about in project 1 running in following cell. What does this tell you about the colab firewall?

In [ ]:
import socket

s = socket.socket(type=socket.SOCK_DGRAM)
s.sendto('hello from colab'.encode(), ('niagaracomputing.org', 12000))
rx, addr = s.recvfrom(2048)
print(rx.decode())




             ,-"""""""-.
            / o       o \ 
           /   \     /   \ 
          /     )---(     \ 
         /     ( 6 6 )     \ 
        /       | " |       \ 
       /         )=(         \ 
      /   o   .--"-"--.   o   \ 
     /    I  /  -   -  \  I    \ 
 .--(    (_}y/\       /\y{_)    )--.
(    ".___l\/__\_____/__\/l___."    )
 \                                 /
  "-._      o O O o O O o      _,-"
      `--Y--.___________.--Y--'
         |==.___________.==| 
         `==.___________.=='

MESSAGE RECEIVED... 



Colab notebooks only allow one cell to be running at a time (this was not always the case). Thus, you can't run a listener in one cell and a sender in another.

## 2. TCP connections

Recall that TCP is a *transport layer* protocol that stands for *transmission control protocol*. Unlike UDP, which sends packets (segments) out without setting up a link, TCP is a *connection-based* protocol. In order to establish a connection, on the server we will use the **listen()** function, and on the client we will use the python **connect(_(host, port)_)** function. Recall that a *host* can be either a domain name, or an IP address.

On your VM (not in colab) try creating tcpserver.py (you can cut and paste the code from the following cell) and then connect to it from colab using the tcpclient.py cell to send a message back and forth:

In [ ]:
#tcpserver.py
# copy this to your VM and run
# ECHO SERVER
import socket

#s = socket.socket(type=socket.SOCK_STREAM)
with socket.socket(type=socket.SOCK_STREAM) as s:
  # SOCK_STREAM means TCP instead of UDP
  s.bind(('', 12000))
  s.listen(1)  # listen for connection, with a backlog of 1
  print('waiting for a connection')
  connection, client_address = s.accept()
  print('connection made from', client_address)
  while True:
    data = connection.recv(1024)
    if data == None: # same as    if not data:
      break
    print('msg received:', data.decode())
    print('sending it back')
    connection.sendall(data)

# no s.close() since the with statement takes care of it

In [ ]:
#tcpclient.py
#fill in your VM's external IP address below and try connecting to it.
# ECHO CLIENT
import socket

HOST = ''    # Replace with your VM server's IP address
PORT = 12000 # The same port as used by the server
with socket.socket() as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', data.decode())

## 3. HTTP Protocol (HTTP/1.1)##

**HTTP** stands for **hyper text transfer protocol**. It specifies the communication rules that a **webclient** (such as chrome, firefox, or edge) and a **webserver** (such as apache2) use to communicate with each other. Typically, the webclient requests a particular webpage, and the webserver sends the html of the page. Sometimes the webclient will send data back to the webserver, for example, when filling in forms on webpages. Recall that **html** stands for hyper text markup language, a language for creating web pages; it is different than http.

### request messages ###
**Request messages** are sent by a client to a target server and have a specific syntax that must be used. (Note: HTTP/2 and HTTP/3 have a different representation for HTTP methods and headers.)

**Request syntax**
A client sends ''request messages'' to the server, which consist of:

1. a **request line**, consisting of the case-sensitive request method, a space , the requested URL, another space, the protocol version, a *carriage return* ('\r' in python), and a *line feed* ('\n' in python), e.g.:

    <code>GET / HTTP/1.1</code>

1. a **HTTP request header field** (at least 1 or more headers in case of HTTP/1.1), each consisting of the case-insensitive field name, a colon, optional leading whitespace, the field value, an optional trailing whitespace and ending with a carriage return ('\r') and a line feed ('\n'), e.g.:

    <code>Host: www.example.com</code>

1. an **empty line**, consisting of a carriage return ('\r') and a line feed ('\n');
1. an optional **HTTP message body**


In the HTTP/1.1 protocol, all header fields except <code>Host: hostname</code> are optional.


In the following code, we use TCP to connect to a webserver directly (on the required port 80), and receive the html directly from the webserver. Try substituting in your VM's external IP address and downloading your own web page.

In python, '\r' represents a carriage return, and '\n' represents a line feed.

Try modifying the code below to talk to your own VM in which we set up the apache2 webserver with your own webpage.

**SUBMISSION: Once you get it working, cut and paste the cell output below into a new document named response.txt. Submit this file in canvas.**


In [ ]:
import socket
import time # for delay

s = socket.socket()
# replace niagaracomputing.org with the external IP address of your server
target = 'niagaracomputing.org'
s.connect((target, 80)) # recall 80 is the required http port #

# replace niagaracomputing.org with the external IP address of your server

s.sendall('GET / HTTP/1.0\r\n'.encode())

#s.sendall('Host: computeraugmented.com\r\n'.encode())
# example target = 'computeraugmented.com'
#s.sendall('Host: target\r\n'.encode()) # DOP! we dont want traget as a str
x = 'Host: ' + target + '\r\n\r\n'
s.sendall(x.encode())

response = s.recv(2048)
print(response.decode())
print('complete')
s.close()

HTTP/1.1 200 OK
Date: Fri, 22 Mar 2024 17:10:36 GMT
Server: Apache/2.4.41 (Ubuntu)
Last-Modified: Mon, 19 Feb 2024 16:51:04 GMT
ETag: "d4-611bee64447c8"
Accept-Ranges: bytes
Content-Length: 212
Vary: Accept-Encoding
Connection: close
Content-Type: text/html

<html>
    <head>
        <title>niagaracomputing.org</title>
    </head>
    <body>
        <h1>niagaracomputing.org</h1>
        <a href="mothershiplog.html">For a good time click here.</a>
    </body>
</html>

complete


TCP, HTTP, IP ADDRESS, PORT, LISTEN, 80, REQUEST, HTML, HEADER, DNS